In [5]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-1]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-1]

In [6]:
Arr = X.values
arr = x.values

import librosa
Arr1 = librosa.feature.delta(Arr)
arr1 = librosa.feature.delta(arr)

Delta1 = pd.DataFrame(Arr1)
delta1 = pd.DataFrame(arr1)

Arr2 = librosa.feature.delta(Arr1)
arr2 = librosa.feature.delta(arr1)

Delta2 = pd.DataFrame(Arr2)
delta2 = pd.DataFrame(arr2)

scaler = Normalizer().fit(Delta1)
Delta1 = scaler.transform(Delta1)
scaler = Normalizer().fit(delta1)
delta1 = scaler.transform(delta1)

Delta1 = np.reshape(Delta1, (Delta1.shape[0],Delta1.shape[1],1))
delta1 = np.reshape(delta1, (delta1.shape[0],delta1.shape[1],1))

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))

scaler = Normalizer().fit(Delta2)
Delta2 = scaler.transform(Delta2)
scaler = Normalizer().fit(delta2)
delta2 = scaler.transform(delta2)

Delta2 = np.reshape(Delta2, (Delta2.shape[0],Delta2.shape[1],1))
delta2 = np.reshape(delta2, (delta2.shape[0],delta2.shape[1],1))

Y = np.array(Y)
y = np.array(y)

lstm_output_size = 70

In [7]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))
Delta2shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

x3 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta2shape)
x3 = MaxPooling1D(pool_length=(2))(x3)
x3 = Dropout(0.5)(x3)

output = concatenate([x1, x2, x3])

x4 = Bidirectional(LSTM(lstm_output_size))(output)
x4 = Dropout(0.1)(x4)
x4 = Dense(1, activation="sigmoid")(x4)

model = Model([Xshape, Delta1shape, Delta2shape],x4)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())

/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  # Remove the CWD from sys.path while we load stuff.
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  del sys.path[0]
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 41, 64)       256         input_1[0][0]                   

In [8]:
history = model.fit([X,Delta1,Delta2], Y, epochs=200, validation_data=([x,delta1,delta2], y))


Train on 175341 samples, validate on 82332 samples
Epoch 1/200
175341/175341 [==============================] - 423s 2ms/step - loss: 0.4159 - accuracy: 0.7674 - val_loss: 0.5503 - val_accuracy: 0.6870
Epoch 2/200
175341/175341 [==============================] - 414s 2ms/step - loss: 0.3763 - accuracy: 0.8125 - val_loss: 0.5570 - val_accuracy: 0.6867
Epoch 3/200
175341/175341 [==============================] - 412s 2ms/step - loss: 0.3665 - accuracy: 0.8192 - val_loss: 0.5342 - val_accuracy: 0.7219
Epoch 4/200
175341/175341 [==============================] - 413s 2ms/step - loss: 0.3622 - accuracy: 0.8223 - val_loss: 0.5716 - val_accuracy: 0.6945
Epoch 5/200
175341/175341 [==============================] - 418s 2ms/step - loss: 0.3598 - accuracy: 0.8247 - val_loss: 0.5468 - val_accuracy: 0.7059
Epoch 6/200
175341/175341 [==============================] - 418s 2ms/step - loss: 0.3576 - accuracy: 0.8263 - val_loss: 0.5407 - val_accuracy: 0.7141
Epoch 7/200
175341/175341 [===============

175341/175341 [==============================] - 396s 2ms/step - loss: 0.2788 - accuracy: 0.8753 - val_loss: 0.5257 - val_accuracy: 0.7351
Epoch 54/200
175341/175341 [==============================] - 392s 2ms/step - loss: 0.2790 - accuracy: 0.8746 - val_loss: 0.5538 - val_accuracy: 0.7321
Epoch 55/200
175341/175341 [==============================] - 397s 2ms/step - loss: 0.2792 - accuracy: 0.8747 - val_loss: 0.5438 - val_accuracy: 0.7339
Epoch 56/200
175341/175341 [==============================] - 396s 2ms/step - loss: 0.2781 - accuracy: 0.8748 - val_loss: 0.5373 - val_accuracy: 0.7359
Epoch 57/200
175341/175341 [==============================] - 394s 2ms/step - loss: 0.2782 - accuracy: 0.8746 - val_loss: 0.5378 - val_accuracy: 0.7307
Epoch 58/200
175341/175341 [==============================] - 395s 2ms/step - loss: 0.2786 - accuracy: 0.8747 - val_loss: 0.5225 - val_accuracy: 0.7310
Epoch 59/200
175341/175341 [==============================] - 397s 2ms/step - loss: 0.2777 - accuracy

175341/175341 [==============================] - 399s 2ms/step - loss: 0.2642 - accuracy: 0.8774 - val_loss: 0.5939 - val_accuracy: 0.7353
Epoch 107/200
175341/175341 [==============================] - 400s 2ms/step - loss: 0.2627 - accuracy: 0.8781 - val_loss: 0.6126 - val_accuracy: 0.7352
Epoch 108/200
175341/175341 [==============================] - 395s 2ms/step - loss: 0.2638 - accuracy: 0.8776 - val_loss: 0.6116 - val_accuracy: 0.7363
Epoch 109/200
175341/175341 [==============================] - 404s 2ms/step - loss: 0.2641 - accuracy: 0.8772 - val_loss: 0.5871 - val_accuracy: 0.7353
Epoch 110/200
175341/175341 [==============================] - 400s 2ms/step - loss: 0.2628 - accuracy: 0.8777 - val_loss: 0.6023 - val_accuracy: 0.7358
Epoch 111/200
175341/175341 [==============================] - 401s 2ms/step - loss: 0.2653 - accuracy: 0.8767 - val_loss: 0.6075 - val_accuracy: 0.7367
Epoch 112/200
175341/175341 [==============================] - 395s 2ms/step - loss: 0.2630 - ac

175341/175341 [==============================] - 206s 1ms/step - loss: 0.2574 - accuracy: 0.8802 - val_loss: 0.6096 - val_accuracy: 0.7353
Epoch 160/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2602 - accuracy: 0.8785 - val_loss: 0.6027 - val_accuracy: 0.7347
Epoch 161/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2586 - accuracy: 0.8795 - val_loss: 0.6219 - val_accuracy: 0.7369
Epoch 162/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2561 - accuracy: 0.8803 - val_loss: 0.6405 - val_accuracy: 0.7375
Epoch 163/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2567 - accuracy: 0.8801 - val_loss: 0.6175 - val_accuracy: 0.7346
Epoch 164/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2555 - accuracy: 0.8806 - val_loss: 0.6343 - val_accuracy: 0.7386
Epoch 165/200
175341/175341 [==============================] - 206s 1ms/step - loss: 0.2579 - ac

In [9]:
from sklearn.metrics import accuracy_score,classification_report
predictions = model.predict([x,delta1,delta2])
predictions = [0 if i<0.5 else 1 for i in predictions]
print("Accuracy: ",accuracy_score(y,predictions))
from sklearn.metrics import confusion_matrix

print("Classification Report: \n",classification_report(y,predictions))
conmat = np.array(confusion_matrix(y, predictions, labels=[0,1]))
confusion = pd.DataFrame(conmat, index=['negative', 'positive'],
                         columns=['predicted_negative','predicted_positive'])
print (confusion)

Accuracy:  0.7377568867512024
Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.45      0.60     37000
           1       0.68      0.98      0.80     45332

    accuracy                           0.74     82332
   macro avg       0.81      0.71      0.70     82332
weighted avg       0.80      0.74      0.71     82332

          predicted_negative  predicted_positive
negative               16469               20531
positive                1060               44272
